### Spark static and dnamic partitioning 

When we run sprk in local and standalone scheduler we use static partitioning. For YARN and Mesos, we use dynamic parttioning. This partitioning has nothing to do with RDD partitions, it is about scaling up and down your cluster dynamically, add and remove executors from the cluster (2015)

### Local mode

1 PC, executor and driver in the same pc (JVM). in the JVM we cache the partitions. The executor has cores to do the tasks. If you have 6 free cores in your pc, you should scale the number of spark core by 2-3 to have 12-18 cores un spark. There is an abstraction between the JVM and the pc cores. You have to avoid have resource competition between tyour tools like hadoop cassandra spark. 

spark-submit --name "myapp" --master local[12] myapp.jar

We can select the number of cores in 3 different ways. The first is in the spark conf file. Second in the spark submit, and third in the code when creating the spark session. The overwrite each other in the same order.

### Standalone

In each machine a worker JVM will start. The spark master JVM and the worker JVM are the resource manager. The driver contacts the spark master asking for cores. The master acts like a scheduler assigning resources to the job. The spark master allocates and schedule resources from the executors. You can use zookeeper to make the master node highly available. In standalone you cant instantiate more than one executor for a single machine. 

### Client Application (yarn)

When we start a client application to the resource manager. The RM will find a node manager to start the application master in a containerin your cluster. The App master  will comunicate with the resource manager for resources. (2:17:00). The application master is also in communication with the client giving feedback to the tasks. We can run mor ethan one app in a spark cluster. The resource manager has an Applications Manager that keeps track of the applications. 

The dirver runs in the client itself. The client might be a laptop. You can use spark interactively. If you call a collect, the RDD will be sent through the network to your driver. The App masters negociate the resources and the resource manager schedules the tasks ans give resources to the app master. 

The problem here is if you remove your pc from the cluster, everything will stop working.

### Cluster mode (yarn)

This does not support spark shells. You can set the number of executors, the number of cores by eecutor and the executor memory. Here Spark uses dyanmic allocation to dyamically increase and decrese the number of executors. This liberates resources when not being used. the applications runs by default in client mode. To change it to cluster, you have to change the deploy mode. 


YARN has its own UI. Details about the applications. 

There is a spark history server. Is  a JVM that runs in a cluster with .5 Gb of ram by default.

![Types of clusters](cap1.png)

- The difference between standalone, yarn and mesos is who starts the executor. In standalone is the Worker JVM who starts.


- The resource manager has track of the available resources (memory, cores, disk)


- If the size of your data in memory is X Mb, once you read it in memory it will be 2-3X Mb. That is the cost of Serialization/Deserialization in java. When you cache it, is less.  


- Everytime you call an action in your spark app, it is a job.A job has 1+ stages and stages have 1+ tasks. 


- spark_local_dirs isin the spark.env allows spark to use different disks in your system. Improves disk spilling.Used for intermediate shuffle data. It is convenient to use SSDs for this. 


- You can allocate cpu resources individually for each machine in the spark.env file SPAR_WORKER_CORES. 


- SPARK_worker_INSTANCES: The number of executors per worker node. 


- SPARK_WORKER_MEMORY: The max ammount of memory you can assign for a worker node. This is distributed between the executors of a worker. 

### from 2:49:00

- Recommended to use at most 75% of a machine memory to spark.

- minium execution heap sze should be 8Gb and max 40Gb

- Memor usage is highly affected by serialization format. Use a fast serialization. This will reduce the cost of garbage collection. 

- rdd.cache() == rdd.persist(MEMORY_ONLY). If the partition does not fit in memory, it wont be stored at all, the will be droped. We would have to reconstruct those partitions with the data lineage. 

- if an executor crashes, the rdd holded by that executor, will be recomputed in another executor by the data lineage. If the executor comes back, it will do another thing. There is no such thing as repliation factor.

- Persist(MEMORY_AND_DISK) sends the older chached partitions to the locla directory. deserialized in memory but serialized on disk. 


- Persist(MEMORY_AND_DISK_SER) Same but serialized in both memory and disk. 


- Persist(DISK) only in disk 

- persist(MEMORY_ONLY_2) replicates usign the memory of 2 jvm. Not convenient. 

- unpersist() remove a partititon forcefully out of memory. When a JVM is full. 

- Dont spill to disk, sometimes it is cheaper to recompute than spill to disk. Dont use replicate unless you really need it. 

- The data is automatically persisted when you during a shuffle opertion. If your program crashes you can go to the last shuffle and start computing from there. 

### Data serialization 3:15:00

- java serializarion is slow but easy. 

- Kryo serializer is faster but some classes are not automatically converted. 

- The cost of garbage collection is proportional to the number of java objects. For example, if we use an array of int instead of  a linked list, it will be faster. 

- We can tune the Garbage collector. Parallel GC by default (works fine). Move to CMS GC or G1 GC for spark streaming. 



### Wide and narrow dependencies 

Calling transformations build a DAG, but once you call an action, the spark driver JVM buidl that DAG with its stage boundaries. The DAG scheduler sends that stages to the task manager. The tasks manager takes the stages and send the tasks to the executors. 


###### Narrow dependencies:  When each partition od the parent RDD is used by at most one partition of the child RDD. 
Map, union, joins when the inputs are co-partitioned (we can set the # of partitions of both RDDs to be the same beor a join to optimize it and hashed with the same function).  

###### Wide dependencies: When multiple child partitions depend on a parent parititon. 
groupBYkey, joins with no co-partition. Here we have to do a shuffle. We have keys in different prtitions and we have to move them accross the cluster. They may produce disk spilling. 

Some transformations use wide transformations inside, groupByKey uses CombineByKey which is a wide trans. 

coalesce never causes a shuffle unless you pass shuffle=True as parameter. Repartition will always create a shuffle. Repartition is useful to rebaance data but keep the same number of partitions or increase it. 

tranformations that use numpartitions as parameter will do a shuffle like distinct. 

When doing mapping and modifying the values, sometimes is useful to pass keepPartitioning=True to avoid moving data and let know spark that the keys wont change. 

Each task is processing one partition. 

![transformations](cap2.png)

#### Accumulators and broadcast variables

Broadcast variables: Used to avoid shuffles in joins when one df fits in memory. Send a read-only table to all the nodes. Similar to distributed cache in hadoop mapreduce.
Spark uses the bittorrent method fot broadcasting. Send a chunk of the file to each node, then they share the chunks within each other moving the network traffic from the driver to the workers. 

Accumulators: Count events during the job execution, commonly used for debugging purposes. How mny lines are blank in a file or the number of corrupt records. Similar to counters in hadoop mapreduce. 

Only the driver program can read the accumulators not the tasks. You can look at the accummulator variables at the Spark UI since the driver program exposes the UI.

Accumulators can be increased or decreased with transformations and actions. 

accum = sc.ccumulator(0)


difference between persist and cache
spark internal talk. 